In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Papillary_Cell_Carcinoma"
cohort = "GSE95425"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Papillary_Cell_Carcinoma/GSE95425"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/GSE95425.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE95425.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/clinical_data/GSE95425.csv"
json_path = "../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Cell-type specific gene programs of the normal human nephron define kidney cancer subtypes"
!Series_summary	"Comprehensive transcriptome studies of cancers often rely on corresponding normal tissue samples to serve as a transcriptional reference. In this study we performed in-depth analyses of normal kidney tissue transcriptomes from TCGA and demonstrate that the histological variability in cellularity, inherent in the kidney architecture, lead to considerable transcriptional differences between samples. This should be considered when comparing expression profiles of normal and cancerous kidney tissues. We exploited these differences to define renal cell-specific gene signatures and used these as framework to analyze renal cell carcinoma (RCC) ontogeny. Chromophobe RCCs express FOXI1-driven genes that define collecting duct intercalated cells whereas HNF-regulated genes, specific for proximal tubule cells, are an integral part of clear cell and pa

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Check if gene expression data is available
is_gene_available = True  # As per background information, this seems to be a gene expression study of normal kidney tissue

# 2. Analyze clinical data availability and create conversion functions
trait_row = 2  # 'sampling depth' can be used to categorize tissue samples
age_row = None  # Age information is not available
gender_row = None  # Gender information is not available

def convert_trait(value):
    """Convert sampling depth information to binary values (0 for cortex, 1 for medulla)"""
    if value is None:
        return None
    
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
        
        if "cortex" in value and "medulla" in value:
            # Mixed sample - we'll treat as None since it's neither purely cortex nor medulla
            return None
        elif "cortex" in value:
            return 0  # Cortex
        elif "medulla" in value:
            return 1  # Medulla
    
    return None

# Define empty conversion functions for age and gender since they're not available
def convert_age(value):
    return None

def convert_gender(value):
    return None

# 3. Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features
if trait_row is not None:
    # Assuming clinical_data was previously loaded
    clinical_df = geo_select_clinical_features(
        clinical_data,
        trait="Sampling_Depth",
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview and save the clinical data
    print("Clinical Data Preview:")
    print(preview_df(clinical_df))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical data to CSV
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical Data Preview:
{'GSM2510512': [0.0], 'GSM2510513': [nan], 'GSM2510514': [1.0], 'GSM2510515': [1.0], 'GSM2510516': [0.0], 'GSM2510517': [0.0], 'GSM2510518': [0.0], 'GSM2510519': [0.0], 'GSM2510520': [0.0], 'GSM2510521': [1.0], 'GSM2510522': [1.0], 'GSM2510523': [1.0], 'GSM2510524': [0.0], 'GSM2510525': [0.0], 'GSM2510526': [0.0], 'GSM2510527': [1.0], 'GSM2510528': [0.0], 'GSM2510529': [0.0], 'GSM2510530': [1.0], 'GSM2510531': [1.0], 'GSM2510532': [1.0], 'GSM2510533': [0.0], 'GSM2510534': [1.0], 'GSM2510535': [1.0], 'GSM2510536': [nan], 'GSM2510537': [1.0], 'GSM2510538': [1.0], 'GSM2510539': [0.0], 'GSM2510540': [0.0], 'GSM2510541': [0.0], 'GSM2510542': [1.0], 'GSM2510543': [1.0], 'GSM2510544': [0.0], 'GSM2510545': [nan], 'GSM2510546': [1.0], 'GSM2510547': [1.0], 'GSM2510548': [0.0], 'GSM2510549': [nan], 'GSM2510550': [1.0], 'GSM2510551': [1.0], 'GSM2510552': [0.0], 'GSM2510553': [1.0], 'GSM2510554': [0.0], 'GSM2510555': [1.0], 'GSM2510556': [1.0], 'GSM2510557': [0.0], 'GSM251055

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Cell-type specific gene programs of the normal human nephron define kidney cancer subtypes"
Line 1: !Series_geo_accession	"GSE95425"
Line 2: !Series_status	"Public on Aug 07 2017"
Line 3: !Series_submission_date	"Feb 27 2017"
Line 4: !Series_last_update_date	"Sep 19 2019"
Line 5: !Series_pubmed_id	"28793269"
Line 6: !Series_summary	"Comprehensive transcriptome studies of cancers often rely on corresponding normal tissue samples to serve as a transcriptional reference. In this study we performed in-depth analyses of normal kidney tissue transcriptomes from TCGA and demonstrate that the histological variability in cellularity, inherent in the kidney architecture, lead to considerable transcriptional differences between samples. This should be considered when comparing expression profiles of normal and cancerous kidney tissues. We exploited these differences to define renal cell-specific gene signatures and used these as framework 

### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers from the extracted data
# These identifiers (ILMN_*) are Illumina BeadArray probe IDs, not standard gene symbols
# They need to be mapped to human gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1498611 rows

Gene annotation preview (first few rows):
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for mapping identifiers to gene symbols
# From the gene annotation data, we can see:
# - 'ID' column contains the ILMN_* identifiers (same as in gene expression data)
# - 'Symbol' column contains gene symbols

# 2. Create the gene mapping dataframe
print("Creating gene mapping from probes to gene symbols...")
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
print(f"Created mapping with {len(mapping_df)} rows")
print("First few rows of mapping data:")
print(preview_df(mapping_df))

# 3. Apply the gene mapping to convert probe-level expression to gene-level expression
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Converted to gene expression data with {len(gene_data.index)} genes")
print("First few gene symbols:")
print(gene_data.index[:10].tolist())

# Save gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Creating gene mapping from probes to gene symbols...
Created mapping with 44837 rows
First few rows of mapping data:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}

Converting probe-level measurements to gene expression data...
Converted to gene expression data with 17999 genes
First few gene symbols:
['A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


Gene expression data saved to ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE95425.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Check if gene_data is empty before normalization
    if gene_data.empty:
        print("Gene data is empty. Skipping normalization.")
        normalized_gene_data = gene_data
    else:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data (even if empty, to maintain consistent workflow)
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# 2. Load and prepare the clinical data
print("\nLoading and preparing clinical data...")
try:
    # Load the clinical data from file
    clinical_df = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
    
    # The trait name used when extracting clinical features in Step 2 was "Sampling_Depth"
    trait_column_name = "Sampling_Depth"
    
    # Check if the data is in the expected format
    if clinical_df.shape[0] == 1:
        # Data is in format with one row and samples as columns
        # Transpose to have samples as rows
        clinical_df = clinical_df.T
        clinical_df.columns = [trait_column_name]
        print(f"Transposed clinical data to shape: {clinical_df.shape}")
    
    print("Clinical data structure:")
    print(clinical_df.head())
    is_trait_available = True
except Exception as e:
    print(f"Error loading clinical data: {e}")
    clinical_df = pd.DataFrame()
    is_trait_available = False

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not normalized_gene_data.empty:
        # Get sample IDs from both datasets
        gene_samples = normalized_gene_data.columns.tolist()
        clinical_samples = clinical_df.index.tolist()
        
        print(f"Gene data has {len(gene_samples)} samples")
        print(f"Clinical data has {len(clinical_samples)} samples")
        
        # Find common samples
        common_samples = set(gene_samples).intersection(set(clinical_samples))
        print(f"Common samples between datasets: {len(common_samples)}")
        
        if len(common_samples) > 0:
            # Prepare genetic data for merging (samples as rows)
            gene_data_T = normalized_gene_data.T
            
            # Merge clinical and genetic data using common samples
            linked_data = clinical_df.join(gene_data_T, how='inner')
            print(f"Linked data shape: {linked_data.shape}")
            
            # Ensure the trait column exists
            if trait_column_name in linked_data.columns:
                # 4. Handle missing values
                print("\nHandling missing values...")
                linked_data = handle_missing_values(linked_data, trait_column_name)
                print(f"After handling missing values, data shape: {linked_data.shape}")
                
                # 5. Determine whether the trait and demographic features are biased
                print("\nChecking for bias in features...")
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait_column_name)
            else:
                print(f"Error: Trait column '{trait_column_name}' not found in linked data.")
                is_biased = True
        else:
            print("Error: No common samples between clinical and gene data.")
            linked_data = pd.DataFrame()
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing.")
        linked_data = pd.DataFrame()
        is_biased = True
except Exception as e:
    print(f"Error in linking data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 6. Final quality validation
print("\nPerforming final validation...")
note = ""
if 'linked_data' in locals() and linked_data.empty:
    note = "Failed to link clinical and genetic data. No common samples or error in processing."
elif 'is_biased' in locals() and is_biased:
    note = "Dataset passed initial processing but contains severely biased trait distribution."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=not normalized_gene_data.empty,
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note=note
)

# 7. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...
After normalization: 17405 unique gene symbols


Normalized gene expression data saved to ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/gene_data/GSE95425.csv

Loading and preparing clinical data...
Loaded clinical data with shape: (1, 53)
Transposed clinical data to shape: (53, 1)
Clinical data structure:
            Sampling_Depth
GSM2510512             0.0
GSM2510513             NaN
GSM2510514             1.0
GSM2510515             1.0
GSM2510516             0.0

Linking clinical and genetic data...
Gene data has 53 samples
Clinical data has 53 samples
Common samples between datasets: 53
Linked data shape: (53, 17406)

Handling missing values...


After handling missing values, data shape: (47, 17406)

Checking for bias in features...
For the feature 'Sampling_Depth', the least common label is '0.0' with 23 occurrences. This represents 48.94% of the dataset.
The distribution of the feature 'Sampling_Depth' in this dataset is fine.


Performing final validation...


Linked data saved to ../../output/preprocess/Kidney_Papillary_Cell_Carcinoma/GSE95425.csv
